In [1]:
import pandas as pd
import requests
import cv2
import requests
from io import BytesIO
from PIL import Image
import numpy as np

In [2]:
df = pd.read_csv("/content/drive/MyDrive/A2_Data.csv")

In [3]:
df

,Unnamed: 0,Image,Review Text
0,3452,['https://images-na.ssl-images-amazon.com/imag...,Loving these vintage springs on my vintage str...
1,1205,['https://images-na.ssl-images-amazon.com/imag...,Works great as a guitar bench mat. Not rugged ...
2,1708,['https://images-na.ssl-images-amazon.com/imag...,We use these for everything from our acoustic ...
3,2078,['https://images-na.ssl-images-amazon.com/imag...,Great price and good quality. It didn't quite...
4,801,['https://images-na.ssl-images-amazon.com/imag...,I bought this bass to split time as my primary...
...,...,...,...
995,1265,['https://images-na.ssl-images-amazon.com/imag...,Extremely impressed with this kit.
996,1882,['https://images-na.ssl-images-amazon.com/imag...,This is a great stereo reverb with plenty of c...
997,1547,['https://images-na.ssl-images-amazon.com/imag...,I really like the simplicity of this bridge. I...
998,1004,['https://images-na.ssl-images-amazon.com/imag...,"Great Product, but there is no warranty in the..."


In [4]:
product_data = {}

for index, row in df.iterrows():
    product_id = row['Unnamed: 0']
    image_urls = eval(row['Image'])
    review_text = row['Review Text']

    # Initialize an empty list to store image data
    image_data = []

    for url in image_urls:
        response = requests.get(url)
        if response.status_code == 200:
            # Convert image data into PIL Image object
            img = Image.open(BytesIO(response.content))
            image_data.append(img)

    # Check if any image data is available
    if image_data:
        product_data[product_id] = (image_data, review_text)
    else:
        df.drop(index,inplace=True)

In [5]:
len(product_data)

994

In [6]:
product_data

{3452: ([<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=117x88>],
  'Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.'),
 1205: ([<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=88x88>,
   <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=66x88>,
   <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=88x88>],
  "Works great as a guitar bench mat. Not rugged enough for abuse but if you take care of it, it will take care of you. Makes organization of workspace much easier because screws won't roll around. Color is good too."),
 1708: ([<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=117x88>],
  "We use these for everything from our acoustic bass down to our ukuleles. I know there is a smaller model available for ukes, violins, etc.; we haven't yet ordered those, but these will work on smaller instruments if one do

In [7]:
from PIL import ImageEnhance, ImageOps
import random

for product_id, (image_data, review_text) in product_data.items():
    preprocessed_images = []
    for img in image_data:
        # Altering contrast
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(1.5)  # Adjust the contrast factor as needed

        # Resizing
        img = img.resize((224, 224))  # Resize to your desired size

        # Geometrical orientation
        img = ImageOps.mirror(img)  # Flip horizontally (replace with desired operation)

        # Random flips
        if random.random() < 0.5:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)

        # Brightness and exposure
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(1.2)  # Adjust the brightness factor as needed

        enhancer = ImageEnhance.Color(img)
        img = enhancer.enhance(0.8)  # Adjust the exposure factor as needed

        preprocessed_images.append(img)

    product_data[product_id] = (preprocessed_images, review_text)

In [8]:
len(product_data)

994

In [9]:
product_data

{3452: ([<PIL.Image.Image image mode=RGB size=224x224>],
  'Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.'),
 1205: ([<PIL.Image.Image image mode=RGB size=224x224>,
   <PIL.Image.Image image mode=RGB size=224x224>,
   <PIL.Image.Image image mode=RGB size=224x224>],
  "Works great as a guitar bench mat. Not rugged enough for abuse but if you take care of it, it will take care of you. Makes organization of workspace much easier because screws won't roll around. Color is good too."),
 1708: ([<PIL.Image.Image image mode=RGB size=224x224>],
  "We use these for everything from our acoustic bass down to our ukuleles. I know there is a smaller model available for ukes, violins, etc.; we haven't yet ordered those, but these will work on smaller instruments if one doesn't extend the feet to their maximum width. They're gentle on the instruments, a

In [10]:
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

# Load the pre-trained MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

17225924/17225924 [==============================] - 0s 0us/step


In [11]:
import numpy as np

X_train = []
for product_id, (image_data, review_text) in product_data.items():
    for img in image_data:
        # Preprocess the image
        img_array = np.array(img)
        img_array = preprocess_input(img_array)
        X_train.append(img_array)

X_train = np.array(X_train)

In [12]:
X_train

array([[[[ 0.30980396,  0.5372549 ,  0.7019608 ],
         [ 0.082353  ,  0.28627455,  0.45882356],
         [-0.372549  , -0.1607843 , -0.02745098],
         ...,
         [ 0.96862745,  0.8117647 ,  0.6313726 ],
         [ 0.96862745,  0.85882354,  0.6784314 ],
         [ 0.9764706 ,  0.8901961 ,  0.70980394]],

        [[ 0.30980396,  0.52156866,  0.69411767],
         [ 0.082353  ,  0.28627455,  0.45098042],
         [-0.372549  , -0.1607843 , -0.02745098],
         ...,
         [ 0.9529412 ,  0.73333335,  0.56078434],
         [ 0.96862745,  0.84313726,  0.6627451 ],
         [ 0.9764706 ,  0.8980392 ,  0.7176471 ]],

        [[ 0.33333337,  0.54509807,  0.7019608 ],
         [ 0.09019613,  0.30196083,  0.45098042],
         [-0.38039213, -0.16862744, -0.02745098],
         ...,
         [ 0.7647059 ,  0.49803925,  0.32549024],
         [ 0.9607843 ,  0.79607844,  0.60784316],
         [ 0.9843137 ,  0.92156863,  0.7411765 ]],

        ...,

        [[-0.7647059 , -0.8039216 , -0

In [13]:
for layer in base_model.layers:
    print(layer.name)

input_1
conv1
conv1_bn
conv1_relu
conv_dw_1
conv_dw_1_bn
conv_dw_1_relu
conv_pw_1
conv_pw_1_bn
conv_pw_1_relu
conv_pad_2
conv_dw_2
conv_dw_2_bn
conv_dw_2_relu
conv_pw_2
conv_pw_2_bn
conv_pw_2_relu
conv_dw_3
conv_dw_3_bn
conv_dw_3_relu
conv_pw_3
conv_pw_3_bn
conv_pw_3_relu
conv_pad_4
conv_dw_4
conv_dw_4_bn
conv_dw_4_relu
conv_pw_4
conv_pw_4_bn
conv_pw_4_relu
conv_dw_5
conv_dw_5_bn
conv_dw_5_relu
conv_pw_5
conv_pw_5_bn
conv_pw_5_relu
conv_pad_6
conv_dw_6
conv_dw_6_bn
conv_dw_6_relu
conv_pw_6
conv_pw_6_bn
conv_pw_6_relu
conv_dw_7
conv_dw_7_bn
conv_dw_7_relu
conv_pw_7
conv_pw_7_bn
conv_pw_7_relu
conv_dw_8
conv_dw_8_bn
conv_dw_8_relu
conv_pw_8
conv_pw_8_bn
conv_pw_8_relu
conv_dw_9
conv_dw_9_bn
conv_dw_9_relu
conv_pw_9
conv_pw_9_bn
conv_pw_9_relu
conv_dw_10
conv_dw_10_bn
conv_dw_10_relu
conv_pw_10
conv_pw_10_bn
conv_pw_10_relu
conv_dw_11
conv_dw_11_bn
conv_dw_11_relu
conv_pw_11
conv_pw_11_bn
conv_pw_11_relu
conv_pad_12
conv_dw_12
conv_dw_12_bn
conv_dw_12_relu
conv_pw_12
conv_pw_12_bn
conv_pw

In [14]:
from keras.models import Model

# Create a new model by removing the top layers
feature_extractor = Model(inputs=base_model.input, outputs=base_model.get_layer('conv_pw_13_bn').output)

# Extract features from the images
features = feature_extractor.predict(X_train)

52/52 [==============================] - 72s 1s/step


In [15]:
features

array([[[[-3.77121019e+00, -1.19916987e+00, -8.80918503e-01, ...,
          -7.69504166e+00, -4.54844761e+00, -7.50827122e+00],
         [-8.72374344e+00, -5.19253731e-01,  1.18771124e+00, ...,
          -9.08765030e+00, -7.94163227e-01, -7.11767006e+00],
         [-6.38624716e+00, -5.24091721e-01, -4.10819530e-01, ...,
          -8.44124985e+00,  5.45519829e+00, -1.99890685e+00],
         ...,
         [-3.93276215e-02,  1.12253189e-01, -2.39061832e+00, ...,
          -1.19089108e+01,  7.34271431e+00, -1.83625221e-02],
         [ 2.07852983e+00, -3.33061290e+00, -3.09519410e+00, ...,
          -1.42882986e+01,  7.97354126e+00,  1.25012732e+00],
         [ 2.57914114e+00, -5.56276417e+00, -1.42901802e+00, ...,
          -7.98957872e+00,  3.95288467e-01, -8.73876286e+00]],

        [[-5.54271412e+00, -2.99549150e+00,  9.69116211e-01, ...,
          -8.41094971e+00, -2.25110888e+00, -7.73288012e+00],
         [-7.53999043e+00, -5.76036930e+00,  1.57478189e+00, ...,
          -1.00871048e

In [16]:
len(features)

1640

In [17]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Flatten the features
features_flat = features.reshape(features.shape[0], -1)

# Normalize the extracted features
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features_flat)

In [18]:
features_normalized.shape

(1640, 50176)

In [19]:
features_normalized

array([[ 0.03043134,  1.1196791 ,  0.3211491 , ..., -0.09472582,
         0.60149854, -0.89033943],
       [-1.9736632 ,  0.04708874,  0.04475336, ...,  0.46424744,
        -1.2917824 , -0.9049609 ],
       [-0.10629171, -0.76379985,  0.16681975, ...,  0.29980087,
         0.74991864,  1.5467017 ],
       ...,
       [ 0.5366127 ,  0.20837645, -0.90195745, ...,  0.88082355,
        -0.18696316,  2.2534387 ],
       [-1.4612255 ,  0.61441624,  1.7065635 , ..., -0.2263247 ,
         1.241432  ,  1.1774133 ],
       [-0.86491156,  0.43521184, -0.7889755 , ...,  0.9329761 ,
        -0.19539413,  1.0873686 ]], dtype=float32)

In [20]:
len(features_normalized)

1640

In [21]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

def preprocess_text(text):
    # Convert to string if necessary
    if not isinstance(text, str):
        text = str(text)

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Preprocess the text reviews
preprocessed_reviews = []
for product_id, (image_data, review_text) in product_data.items():
    preprocessed_review = preprocess_text(review_text)
    preprocessed_reviews.append(preprocessed_review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [22]:
preprocessed_reviews

['love vintag spring vintag strat good tension great stabil float bridg want spring way go',
 'work great guitar bench mat rug enough abus take care take care make organ workspac much easier screw wont roll around color good',
 'use everyth acoust bass ukulel know smaller model avail uke violin etc havent yet order work smaller instrument one doesnt extend foot maximum width theyr gentl instrument grippi materi keep secur greatest benefit write music comput need set guitar use keyboardmous easier hang stand sever gave one friend christma well ive use mine stage fold small enough fit right gig bag',
 'great price good qualiti didnt quit match radiu sound hole close enough',
 'bought bass split time primari bass dean edg might win bass boost outstand activ pickup realli allow adjust sound want recommend anyon your beginn like long ago excel bass start your tour andor music make money bass beati stage color bit darker pictur around great buy',
 'toy side instrument side made indonesia pro

In [23]:
len(preprocessed_reviews)

994

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer object
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed reviews
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_reviews)

In [25]:
tfidf_matrix

<994x5024 sparse matrix of type '<class 'numpy.float64'>'
	with 32760 stored elements in Compressed Sparse Row format>

In [26]:
product_data[1306][1]

'This product is good and is used in professional Mike Date .. fast shipping and worth buying'

In [39]:
from collections import defaultdict
import math

# Step 1: Compute Term Frequency (TF) for each term in each document
tf_scores = defaultdict(dict)
for product_id, (image_data, review_text) in product_data.items():
    # review_text
    terms = str(review_text).split()
    total_terms = len(terms)
    term_freq = defaultdict(int)
    for term in terms:
        term_freq[term] += 1
    for term, freq in term_freq.items():
        tf_scores[product_id][term] = freq / total_terms

# Step 2: Compute Inverse Document Frequency (IDF) for each term
num_documents = len(product_data)
idf_scores = defaultdict(float)
for product_id, (image_data, review_text) in product_data.items():
    terms = set(str(review_text).split())
    for term in terms:
        idf_scores[term] += 1

for term, count in idf_scores.items():
    idf_scores[term] = math.log(num_documents / count)

# Step 3: Compute TF-IDF scores
tfidf_scores = defaultdict(dict)
for product_id, tf_score in tf_scores.items():
    for term, tf in tf_score.items():
        tfidf_scores[product_id][term] = tf * idf_scores[term]

# Let's print out TF-IDF scores for the first document for demonstration
first_product_id = list(product_data.keys())[0]
print(f"TF-IDF scores for the first document (product_id: {first_product_id}):")
for term, score in tfidf_scores[first_product_id].items():
    print(f"{term}: {score}")



TF-IDF scores for the first document (product_id: 3452):
Loving: 0.16779973043504404
these: 0.09451057363020515
vintage: 0.23442096482135338
springs: 0.31368242799937646
on: 0.022521935632769295
my: 0.018035850836167638
strat.: 0.138107506416987
They: 0.06710531348270206
have: 0.028056919758337216
a: 0.0078659773496977
good: 0.04987814598130449
tension: 0.14303511606006686
and: 0.013592332868602819
great: 0.042405903064627166
stability.: 0.18653343801774527
If: 0.06583321859464838
you: 0.031753769164037215
are: 0.06677354154292658
floating: 0.15684121399968823
your: 0.10791688801179161
bridge: 0.09460918445930862
want: 0.07542873898603414
the: 0.014296171641736055
most: 0.07726178213032496
out: 0.05202712065502702
of: 0.018409379366400944
than: 0.05022465107398183
way: 0.07974306724527425
to: 0.010985720289717997
go.: 0.11520756424435988


In [ ]:
pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552449 sha256=82b5a0ab6ca53f2eecbae51a2f0f3ba28759ce3ac5b659b85dfd9c5bcba11e24
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [40]:
features.shape

(1640, 7, 7, 1024)

In [41]:

import requests
from io import BytesIO
from PIL import Image
from keras.preprocessing import image as keras_image  # Import image module from Keras

# Load and preprocess your input image from URL
input_img_url = 'https://images-na.ssl-images-amazon.com/images/I/71mKngihFdL._SY88.jpg'  # Replace with the URL from your CSV file

response = requests.get(input_img_url)
input_img = Image.open(BytesIO(response.content))
input_img = input_img.resize((224, 224))  # Resize to the required dimensions
input_img_array = keras_image.img_to_array(input_img)
input_img_array = preprocess_input(input_img_array)
input_img_array = input_img_array.reshape(1, 224, 224, 3)  # Reshape for model input

# Extract features using the feature extractor model (MobileNet in this case)
input_image_features = feature_extractor.predict(input_img_array)


1/1 [==============================] - 0s 86ms/step


In [46]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_images(input_image_features, all_image_features, top_n=3):
    # Compute cosine similarity between input image features and all other images
    similarity_scores = cosine_similarity(input_image_features.reshape(1, -1), all_image_features)

    # Get indices of top similar images
    similar_indices = similarity_scores.argsort()[0][-top_n-1:-1][::-1]

    return similar_indices, similarity_scores[0][similar_indices]

# Assuming you have extracted features for all images and have them in 'features_normalized'
# Also, assuming 'input_image_features' is the feature vector for your input image

# Find the indices and similarity scores of the most similar images
similar_indices, similarity_scores = find_similar_images(input_image_features, features_normalized)
print(similar_indices)
print(similarity_scores)

# Get the product IDs corresponding to the similar images
similar_product_ids = []
similar_images = []
similar_reviews = []

for i in similar_indices:
    image_array = X_train[i]  # Get the image array from X_train
    # Find the product ID corresponding to the image array in product_data
    for product_id, (images, review_text) in product_data.items():
        if np.array_equal(images[0], image_array):  # Assuming the first image in the list corresponds to the image array
            similar_product_ids.append(product_id)
            similar_reviews.append(review_text)
            similar_images.append(images[0])
            break  # Stop searching once product ID is found
# print(similar_product_ids)
# Display the similar images and their similarity scores
for i, (product_id, review, image, score) in enumerate(zip(similar_product_ids, similar_reviews, similar_images, similarity_scores)):
    print(f"Similar Image {i+1}:")
    print(f"Product ID - {product_id}")
    print(f"Similarity Score - {score}")
    print("Review:", review)
    image.show()

for i in range(len(similar_indices)):
    print(f'Image with index {similar_indices[i]} from the X_train array has similarity index of {similarity_scores[i]}')
    # print(f'Review:{similar_reviews[i]}')


[ 987 1476 1163]
[0.2831053  0.23505646 0.22264925]
[]
[]
Image with index 987 from the X_train array has similarity index of 0.2831053137779236
Image with index 1476 from the X_train array has similarity index of 0.23505645990371704
Image with index 1163 from the X_train array has similarity index of 0.2226492464542389


In [31]:
from PIL import Image
import numpy as np

# Assuming your array is named 'image_array'
image_array = X_train[1163]
print(image_array)
# Rescale the image array from the range [-1, 1] to [0, 255]
rescaled_array = ((image_array + 1) / 2) * 255

# Convert the rescaled array to uint8 type
rescaled_array = rescaled_array.astype(np.uint8)

# Create an image from the array
image = Image.fromarray(rescaled_array)
image.save("output_image_2.png")

[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 ...

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]


In [32]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from keras.preprocessing import image as keras_image

# Assuming you have the input image and review
input_image_path = 'https://images-na.ssl-images-amazon.com/images/I/618I1bEQ7OL._SY88.jpg'  # Replace with the path to your input image
input_review = "I had read another review in here from someone using this with the MXL V67G after popping out the existing mounting ring with a flat blade screwdriver, so I took a chance and ordered one for myself.  That guy was 100% correct! I just popped out the existing mounting ring with a flat blade and my V67G slipped right in and could be tighly secured using the black mounting ring supplied with the V67G.I had read a lot of negative reviews about the MXL shockmount for this microphone so I didn't want that one.  This shockmount bears a striking resemblance to the MXL 890 shockmount that came standard with my 890 vocal mic and with the same quality of construction.  The price is definitely more than fair for what you get here!So... If you have an MXL V67G order with confidence!  I would say that this would be a much better choice than the MXL alternative.I was also blown away how fast Amazon got this thing to me!  I ordered it last night and it showed up on my front porch this afternoon!"

# Load and preprocess the input image
response = requests.get(input_img_url)
input_img = Image.open(BytesIO(response.content))
input_img = input_img.resize((224, 224))  # Resize to the required dimensions
input_img_array = keras_image.img_to_array(input_img)
input_img_array = preprocess_input(input_img_array)
input_img_array = input_img_array.reshape(1, 224, 224, 3)  # Reshape for model input

# Extract features using the feature extractor model (MobileNet in this case)
input_image_features = feature_extractor.predict(input_img_array)

# Calculate cosine similarity between the input image and all other images
image_similarity_scores = cosine_similarity(input_image_features.reshape(1, -1), features_normalized)

# Get the indices of top similar images
top_image_indices = image_similarity_scores.argsort()[0][-3:][::-1]

# Save the results using pickle
image_similarity_results = {
    "input_image_path": input_image_path,
    "input_image_features": input_image_features,
    "top_image_indices": top_image_indices
}

with open('image_similarity_results_1.pkl', 'wb') as f:
    pickle.dump(image_similarity_results, f)

1/1 [==============================] - 0s 72ms/step


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming you have the preprocessed_reviews array and input_review
preprocessed_reviews.append(preprocess_text(input_review))

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_reviews)

# Calculate cosine similarity between the input review and all other reviews
review_similarity_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

# Get the indices of top similar reviews
top_review_indices = review_similarity_scores.argsort()[0][-3:][::-1]

# Save the results using pickle
review_similarity_results = {
    "input_review": input_review,
    "top_review_indices": top_review_indices
}

with open('review_similarity_results.pkl', 'wb') as f:
    pickle.dump(review_similarity_results, f)


In [34]:
# Load image similarity results
with open('image_similarity_results_1.pkl', 'rb') as f:
    image_similarity_results = pickle.load(f)

# Load review similarity results
with open('review_similarity_results.pkl', 'rb') as f:
    review_similarity_results = pickle.load(f)

print("Shape of image_similarity_scores:", image_similarity_scores.shape)
print("Shape of review_similarity_scores:", review_similarity_scores.shape)

# Retrieve similarity scores and indices
image_indices = image_similarity_results["top_image_indices"]
review_indices = review_similarity_results["top_review_indices"]

print("Image indices:", image_indices)
print("Review indices:", review_indices)
# print(review_similarity_scores[0][32])
# Calculate composite similarity scores
composite_similarity_scores = []
for img_index, rev_index in zip(image_indices, review_indices):
    print("Image index:", img_index)
    print("Review index:", rev_index)
    img_similarity = image_similarity_scores[0][img_index]
    rev_similarity = review_similarity_scores[0][rev_index]
    composite_similarity = (img_similarity + rev_similarity) / 2  # Average similarity score
    composite_similarity_scores.append(composite_similarity)

# Combine indices for ranking
combined_indices = list(zip(image_indices, review_indices))

# Rank the pairs based on composite similarity scores
ranked_pairs = sorted(zip(combined_indices, composite_similarity_scores), key=lambda x: x[1], reverse=True)

# Display ranked pairs
for i, ((img_index, rev_index), score) in enumerate(ranked_pairs, 1):
    print(f"Rank {i}: Image Index - {img_index}, Review Index - {rev_index}, Composite Similarity Score - {score}")


Shape of image_similarity_scores: (1, 1640)
Shape of review_similarity_scores: (1, 994)
Image indices: [ 986  987 1476]
Review indices: [ 32 314 867]
Image index: 986
Review index: 32
Image index: 987
Review index: 314
Image index: 1476
Review index: 867
Rank 1: Image Index - 986, Review Index - 32, Composite Similarity Score - 0.7913028377198661
Rank 2: Image Index - 987, Review Index - 314, Composite Similarity Score - 0.26305014400851784
Rank 3: Image Index - 1476, Review Index - 867, Composite Similarity Score - 0.2029801013151461
